In [1]:
import numpy as np
import pandas as pd
import matplotlib
import os
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from abc import ABCMeta, abstractmethod
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [2]:
def read_data(path):
    data = pd.read_csv(path)
    return data

In [3]:
path = os.path.realpath(os.path.join(os.getcwd() , '..', 'data_for_training', 'BEV_data_w_hpi.csv'))

In [4]:
data = read_data(path)
if "Unnamed: 0.1" in data.columns:
    data = data.drop(["Unnamed: 0.1"], axis = 1)

In [5]:
print(data.info(),data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zip code                    402 non-null    int64  
 1   number_registration         402 non-null    int64  
 2   Population                  402 non-null    int64  
 3   household count             402 non-null    int64  
 4   <10,000                     402 non-null    float64
 5   10,000~14,999               402 non-null    float64
 6   15,000~24,999               402 non-null    float64
 7   25,000~34,999               402 non-null    float64
 8   35,000~49,999               402 non-null    float64
 9   50,000~74,999               402 non-null    float64
 10  75,000~99,999               402 non-null    float64
 11  100,000~149,999             402 non-null    float64
 12  150,000~199,999             402 non-null    float64
 13  >200,000                    402 non

In [6]:
data = data[data['housing_price_index'] != "."]
data['housing_price_index'] = data['housing_price_index'].astype(float)
print(data.info(),data.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 401
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zip code                    401 non-null    int64  
 1   number_registration         401 non-null    int64  
 2   Population                  401 non-null    int64  
 3   household count             401 non-null    int64  
 4   <10,000                     401 non-null    float64
 5   10,000~14,999               401 non-null    float64
 6   15,000~24,999               401 non-null    float64
 7   25,000~34,999               401 non-null    float64
 8   35,000~49,999               401 non-null    float64
 9   50,000~74,999               401 non-null    float64
 10  75,000~99,999               401 non-null    float64
 11  100,000~149,999             401 non-null    float64
 12  150,000~199,999             401 non-null    float64
 13  >200,000                    401 non

In [7]:
#  Description of the dataset
Description_data = {'Variable': ['zip code',
                                 'number_registration',
                                 'Population',
                                 'household count',
                                 '<10,000',
                                 '10,000~14,999',
                                 '15,000~24,999',
                                 '25,000~34,999',
                                 '35,000~49,999',
                                 '50,000~74,999',
                                 '75,000~99,999',
                                 '100,000~149,999',
                                 '150,000~199,999',
                                 '>200,000',
                                 'EV charging station number',
                                 'housing_price_index'],
                    
                    'Description': ['United States Zip Codes',
                                    'The number of electric vehicle registrations',
                                    'The number of Population',
                                    'The number of household',
                                    'The income less than \$10,000',
                                    'The income from \$10,000 to \$14,999',
                                    'The income from \$15,000 to \$24,999',
                                    'The income from \$25,000 to \$34,999',
                                    'The income from \$35,000 to \$49,999',
                                    'The income from \$50,000 to \$74,999',
                                    'The income from \$75,000 to \$99,999',
                                    'The income from \$100,000 to \$149,999',
                                    'The income from \$150,000 to \$199,999',
                                    'The income more than \$200,000',
                                    'EV charging station number',
                                    'housing price index'
                                   ]}

Description_table = pd.DataFrame(data=Description_data, index=['zip code',
                                                               'number_registration',
                                                               'Population',
                                                               'household count',
                                                               '<10,000',
                                                               '10,000~14,999',
                                                               '15,000~24,999',
                                                               '25,000~34,999',
                                                               '35,000~49,999',
                                                               '50,000~74,999',
                                                               '75,000~99,999',
                                                               '100,000~149,999',
                                                               '150,000~199,999',
                                                               '>200,000',
                                                               'EV charging station number',
                                                               'housing_price_index'])
Description_table.style.set_properties(**{'font-size': '12pt',}).set_table_styles([{'selector': 'th', 'props': [('font-size', '10pt')]}])

,Variable,Description
zip code,zip code,United States Zip Codes
number_registration,number_registration,The number of electric vehicle registrations
Population,Population,The number of Population
household count,household count,The number of household
"<10,000","<10,000","The income less than \$10,000"
"10,000~14,999","10,000~14,999","The income from \$10,000 to \$14,999"
"15,000~24,999","15,000~24,999","The income from \$15,000 to \$24,999"
"25,000~34,999","25,000~34,999","The income from \$25,000 to \$34,999"
"35,000~49,999","35,000~49,999","The income from \$35,000 to \$49,999"
"50,000~74,999","50,000~74,999","The income from \$50,000 to \$74,999"


In [8]:
data.head()

,zip code,number_registration,Population,household count,"<10,000","10,000~14,999","15,000~24,999","25,000~34,999","35,000~49,999","50,000~74,999","75,000~99,999","100,000~149,999","150,000~199,999",">200,000",EV charging station number,housing_price_index
0,98575,1,142,73,13.7,0.0,8.2,12.3,12.3,19.2,15.1,12.3,2.7,4.1,2,290.76
1,98068,10,256,108,0.0,0.0,2.8,0.0,13.9,24.1,7.4,39.8,12.0,0.0,11,265.53
2,98814,6,376,217,2.3,26.7,4.6,14.3,12.4,23.5,6.0,10.1,0.0,0.0,0,239.77
3,98638,10,1225,493,4.3,5.1,8.9,15.0,13.8,21.9,14.4,10.5,2.2,3.9,0,172.13
4,98605,3,1165,417,5.0,5.3,15.3,8.9,12.7,24.0,13.9,13.4,1.4,0.0,2,366.78


In [9]:
data.columns = [
'Zip_code',
'Number_registration',
'Population',
'Household_count',
'Less_than_10000',
'10000to14999',
'15000to24999',
'25000to34999',
'35000to49999',
'50000to74999',
'75000to99999',
'100000to149999',
'150000to199999',
'More_than_200000',
'EV_charging_station_number',
'Housing_price_index']

In [10]:
data.corr()

,Zip_code,Number_registration,Population,Household_count,Less_than_10000,10000to14999,15000to24999,25000to34999,35000to49999,50000to74999,75000to99999,100000to149999,150000to199999,More_than_200000,EV_charging_station_number,Housing_price_index
Zip_code,1.000000,-0.561016,-0.277970,-0.311419,0.171298,0.204279,0.372115,0.347257,0.316807,0.211791,-0.087757,-0.341612,-0.436738,-0.450676,-0.250223,-0.562351
Number_registration,-0.561016,1.000000,0.669245,0.712012,-0.189882,-0.329909,-0.427142,-0.433263,-0.444101,-0.405768,0.054110,0.396535,0.628664,0.747409,0.417978,0.818852
Population,-0.277970,0.669245,1.000000,0.982414,-0.024906,-0.208173,-0.162490,-0.173954,-0.215144,-0.180945,0.129169,0.166497,0.269823,0.257157,0.258021,0.667812
Household_count,-0.311419,0.712012,0.982414,1.000000,-0.003015,-0.196694,-0.159957,-0.182842,-0.232142,-0.213201,0.107496,0.166112,0.284284,0.289977,0.333925,0.718992
Less_than_10000,0.171298,-0.189882,-0.024906,-0.003015,1.000000,0.301035,0.436702,0.245345,0.001860,-0.222507,-0.216134,-0.359942,-0.369454,-0.287468,0.120802,-0.060388
10000to14999,0.204279,-0.329909,-0.208173,-0.196694,0.301035,1.000000,0.461671,0.267811,0.129444,-0.087573,-0.191899,-0.485574,-0.399071,-0.367360,-0.040851,-0.248053
15000to24999,0.372115,-0.427142,-0.162490,-0.159957,0.436702,0.461671,1.000000,0.398344,0.323770,0.023310,-0.307924,-0.635399,-0.576281,-0.523722,-0.104545,-0.317978
25000to34999,0.347257,-0.433263,-0.173954,-0.182842,0.245345,0.267811,0.398344,1.000000,0.329049,0.167997,-0.315651,-0.573965,-0.502864,-0.512350,-0.153507,-0.346753
35000to49999,0.316807,-0.444101,-0.215144,-0.232142,0.001860,0.129444,0.323770,0.329049,1.000000,0.197414,-0.324381,-0.495094,-0.476191,-0.451652,-0.195757,-0.342392
50000to74999,0.211791,-0.405768,-0.180945,-0.213201,-0.222507,-0.087573,0.023310,0.167997,0.197414,1.000000,-0.102762,-0.168326,-0.440710,-0.454540,-0.214162,-0.317261


In [11]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm
# Choose the features to be used
subcols= [
'Zip_code',
'Population',
'Household_count',
'Less_than_10000',
'10000to14999',
'15000to24999',
'25000to34999',
'35000to49999',
'50000to74999',
'75000to99999',
'100000to149999',
'150000to199999',
'More_than_200000',
'EV_charging_station_number',
'Housing_price_index']
X_train=data[subcols]
y_train=data['Number_registration']

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# Fit the data to the model
model1 = sm.OLS(y_train, X_train).fit() #ordinary least square
print(model1.summary())

                             OLS Regression Results                            
Dep. Variable:     Number_registration   R-squared:                       0.873
Model:                             OLS   Adj. R-squared:                  0.868
Method:                  Least Squares   F-statistic:                     177.0
Date:                 Wed, 23 Nov 2022   Prob (F-statistic):          3.08e-162
Time:                         22:40:35   Log-Likelihood:                -2312.7
No. Observations:                  401   AIC:                             4657.
Df Residuals:                      385   BIC:                             4721.
Df Model:                           15                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const     

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


### To select variables, we compute VIFs.

**Variance Inflation Factor (VIF)**

In [12]:
# calculate Variance Inflation Factor for each explanatory variable
from statsmodels.stats.outliers_influence import variance_inflation_factor

# The dataframe passed to VIF must include the intercept term. We add it the same way we did before.
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

VIF(data, subcols)

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Zip_code                         1.653129
Population                      36.960833
Household_count                 42.659687
Less_than_10000               1675.999960
10000to14999                  1369.212899
15000to24999                  2248.672670
25000to34999                  1951.128460
35000to49999                  2909.538848
50000to74999                  3490.997069
75000to99999                  1977.593229
100000to149999                5006.122747
150000to199999                2125.570786
More_than_200000              4104.125692
EV_charging_station_number       1.442834
Housing_price_index              3.733707
dtype: float64

## We try removing one of the variables with high VIFs. We can remove either '100000to149999' or 'More_than_200000'.

### Direction 1:
#### 1.1 - If we remove '100000to149999':

In [13]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm
# Choose the features to be used
subcols_10000to14999= [
'Zip_code',
'Population',
'Household_count',
'Less_than_10000',
'10000to14999',
'15000to24999',
'25000to34999',
'35000to49999',
'50000to74999',
'75000to99999',
#'100000to149999',
'150000to199999',
'More_than_200000',
'EV_charging_station_number',
'Housing_price_index']
X_train=data[subcols_10000to14999]
y_train=data['Number_registration']

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# Fit the data to the model
model_10000to14999= sm.OLS(y_train, X_train).fit() #ordinary least square
print(model_10000to14999.summary())
print('After remove it,we get VIFs:')
print(VIF(data,subcols_10000to14999))

                             OLS Regression Results                            
Dep. Variable:     Number_registration   R-squared:                       0.872
Model:                             OLS   Adj. R-squared:                  0.867
Method:                  Least Squares   F-statistic:                     188.0
Date:                 Wed, 23 Nov 2022   Prob (F-statistic):          1.52e-162
Time:                         22:40:35   Log-Likelihood:                -2314.7
No. Observations:                  401   AIC:                             4659.
Df Residuals:                      386   BIC:                             4719.
Df Model:                           14                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const     

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


#### 2.2 - Then we will also need to remove the "Household_count"

In [14]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm
# Choose the features to be used
subcols_10000to14999_Household_count= [
'Zip_code',
'Population',
#'Household_count',
'Less_than_10000',
'10000to14999',
'15000to24999',
'25000to34999',
'35000to49999',
'50000to74999',
'75000to99999',
#'100000to149999',
'150000to199999',
'More_than_200000',
'EV_charging_station_number',
'Housing_price_index']
X_train=data[subcols_10000to14999_Household_count]
y_train=data['Number_registration']

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# Fit the data to the model
model_10000to14999_Household_count= sm.OLS(y_train, X_train).fit() #ordinary least square
print(model_10000to14999_Household_count.summary())
print('After remove it,we get VIFs:')
print(VIF(data,subcols_10000to14999_Household_count))

                             OLS Regression Results                            
Dep. Variable:     Number_registration   R-squared:                       0.866
Model:                             OLS   Adj. R-squared:                  0.861
Method:                  Least Squares   F-statistic:                     191.8
Date:                 Wed, 23 Nov 2022   Prob (F-statistic):          1.35e-159
Time:                         22:40:36   Log-Likelihood:                -2324.6
No. Observations:                  401   AIC:                             4677.
Df Residuals:                      387   BIC:                             4733.
Df Model:                           13                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const     

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


### Direction 2:
#### 2.1 - If we remove 'More_than_200000':

In [15]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm
# Choose the features to be used
subcols_More_than_200000= [
'Zip_code',
'Population',
'Household_count',
'Less_than_10000',
'10000to14999',
'15000to24999',
'25000to34999',
'35000to49999',
'50000to74999',
'75000to99999',
'100000to149999',
'150000to199999',
#'More_than_200000',
'EV_charging_station_number',
'Housing_price_index']
X_train=data[subcols_More_than_200000]
y_train=data['Number_registration']

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# Fit the data to the model
model_More_than_200000= sm.OLS(y_train, X_train).fit() #ordinary least square
print(model_More_than_200000.summary())
print('After remove More_than_200000,we get VIFs:')
print(VIF(data,subcols_More_than_200000))

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                             OLS Regression Results                            
Dep. Variable:     Number_registration   R-squared:                       0.872
Model:                             OLS   Adj. R-squared:                  0.867
Method:                  Least Squares   F-statistic:                     187.3
Date:                 Wed, 23 Nov 2022   Prob (F-statistic):          2.71e-162
Time:                         22:40:36   Log-Likelihood:                -2315.3
No. Observations:                  401   AIC:                             4661.
Df Residuals:                      386   BIC:                             4721.
Df Model:                           14                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const     

#### 2.2 - Then we will also need to remove the "Household_count"

In [16]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm
# Choose the features to be used
subcols_More_than_200000_Household_count= [
'Zip_code',
'Population',
#'Household_count',
'Less_than_10000',
'10000to14999',
'15000to24999',
'25000to34999',
'35000to49999',
'50000to74999',
'75000to99999',
'100000to149999',
'150000to199999',
#'More_than_200000',
'EV_charging_station_number',
'Housing_price_index']
X_train=data[subcols_More_than_200000_Household_count]
y_train=data['Number_registration']

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# Fit the data to the model
model_More_than_200000_Household_count= sm.OLS(y_train, X_train).fit() #ordinary least square
print(model_More_than_200000_Household_count.summary())
print('After remove More_than_200000,we get VIFs:')
print(VIF(data,subcols_More_than_200000_Household_count))

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                             OLS Regression Results                            
Dep. Variable:     Number_registration   R-squared:                       0.865
Model:                             OLS   Adj. R-squared:                  0.861
Method:                  Least Squares   F-statistic:                     191.2
Date:                 Wed, 23 Nov 2022   Prob (F-statistic):          2.37e-159
Time:                         22:40:36   Log-Likelihood:                -2325.2
No. Observations:                  401   AIC:                             4678.
Df Residuals:                      387   BIC:                             4734.
Df Model:                           13                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const     

#### 2.3 - Then we will also need to remove the "150000to199999"

In [17]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm
# Choose the features to be used
subcols_More_than_200000_Household_count_150000to199999= [
'Zip_code',
'Population',
#'Household_count',
'Less_than_10000',
'10000to14999',
'15000to24999',
'25000to34999',
'35000to49999',
'50000to74999',
'75000to99999',
'100000to149999',
#'150000to199999',
#'More_than_200000',
'EV_charging_station_number',
'Housing_price_index']
X_train=data[subcols_More_than_200000_Household_count_150000to199999]
y_train=data['Number_registration']

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# Fit the data to the model
model_More_than_200000_Household_count_150000to199999= sm.OLS(y_train, X_train).fit() #ordinary least square
print(model_More_than_200000_Household_count_150000to199999.summary())
print('After remove More_than_200000,we get VIFs:')
print(VIF(data,subcols_More_than_200000_Household_count_150000to199999))

                             OLS Regression Results                            
Dep. Variable:     Number_registration   R-squared:                       0.856
Model:                             OLS   Adj. R-squared:                  0.852
Method:                  Least Squares   F-statistic:                     192.3
Date:                 Wed, 23 Nov 2022   Prob (F-statistic):          5.61e-155
Time:                         22:40:36   Log-Likelihood:                -2338.4
No. Observations:                  401   AIC:                             4703.
Df Residuals:                      388   BIC:                             4755.
Df Model:                           12                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const     

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Zip_code                      1.645613
Population                    1.982199
Less_than_10000               1.521323
10000to14999                  1.563722
15000to24999                  2.147615
25000to34999                  1.776692
35000to49999                  1.770100
50000to74999                  1.373316
75000to99999                  1.513678
100000to149999                3.131940
EV_charging_station_number    1.250114
Housing_price_index           3.277998
dtype: float64
